# Linear Regression


### Initialization

In [ ]:
from sklearn.linear_model import LinearRegression
from utility import start
from utility import train

X_train, X_test, y_train, y_test, df = start("encoded_data_04_13.csv")

In [ ]:
model = LinearRegression()
y_pred, y_pred_original, y_test_original = train(model, X_train, X_test, y_train, y_test)

In [ ]:
z = pd.merge(X_test, df, on=['height_in_cm','year','goals_for','goals_against','goals','assists','red_cards','yellow_cards','minutes_played','age_at_evaluation','country_of_citizenship_encoded','sub_position_encoded','club_id_encoded','domestic_competition_id_encoded'])
print(z.shape)
z.set_index(X_test.index, inplace=True)
z['Predicted_Value_log'] = y_pred
z['Predicted_Value'] = np.power(10, y_pred)

results = z[['name', 'year', 'market_value_in_eur', 'Predicted_Value', 'log_market_value_base10', 'Predicted_Value_log']]
results.sort_values(by='Predicted_Value', ascending=False)



In [ ]:
import matplotlib.pyplot as plt

from matplotlib.ticker import FuncFormatter
# Scatter plot
plt.scatter(y_test_original, y_pred_original, s=20, color='blue', alpha=0.5)  # Set dot size to 20, blue color, and transparency to 0.5
plt.xlabel("Real value")
plt.ylabel("Predicted value")
plt.title("Value vs Predicted value")

# Add diagonal line
plt.plot([min(y_test_original), max(y_test_original)], [min(y_test_original), max(y_test_original)], color='red')
plt.text(min(y_test_original), max(y_pred_original), "Over evaluation", fontsize=10, color='green', verticalalignment='top', horizontalalignment='left')
plt.text(max(y_test_original), min(y_pred_original), "Under evaluation", fontsize=10, color='orange', verticalalignment='bottom', horizontalalignment='right')

# Add player names for values over 50 million
for index, row in results.iterrows():
    if row['market_value_in_eur'] > 110000000 or row['Predicted_Value'] > 410000000:  # Assuming the value is in EUR
        print(row['name'])
        plt.annotate(row['name'], (row['market_value_in_eur'], row['Predicted_Value']), fontsize=6, color='black')

# Initialize lists for names and coordinates
#names = []
#coords = []

# Add player names for values over 100 million to the lists
#for index, row in z.iterrows():
#    if row['market_value_in_eur'] > 120000000:  # Assuming the value is in EUR
#        names.append(row['name'])
##        coords.append((row['market_value_in_eur'], row['Predicted_Value']))

# Annotate names as a text box with arrows
#for name, coord in zip(names, coords):
#    plt.annotate(name, xy=coord, xytext=(0, 0), textcoords='offset points', arrowprops=dict(facecolor='black', arrowstyle='->'), fontsize=6)

# Set formatter for x-axis
def millions_formatter(x, pos):
    return f'{x / 1000000:.0f}M'

formatter = FuncFormatter(millions_formatter)
plt.gca().xaxis.set_major_formatter(formatter)
plt.gca().yaxis.set_major_formatter(formatter)
plt.grid(alpha=0.2, zorder=1)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Scatter plot
plt.scatter(y_test, y_pred, s=20, color='blue', alpha=0.5)  # Set dot size to 20, blue color, and transparency to 0.5
plt.xlabel("Real value")
plt.ylabel("Predicted value")
plt.title("Value vs Predicted value")

# Add diagonal line
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')
plt.text(min(y_test), max(y_pred), "Over evaluation", fontsize=10, color='green', verticalalignment='top', horizontalalignment='left')
plt.text(max(y_test), min(y_pred), "Under evaluation", fontsize=10, color='orange', verticalalignment='bottom', horizontalalignment='right')

plt.show()
